# IDENTIFIKASI SUARA BUKA TUTUP

Tugas ini membahas klasifikasi suara menggunakan fitur MFCC (Mel-Frequency Cepstral Coefficients). Dataset yang digunakan terdiri dari 200 rekaman suara dalam format WAV, dengan 100 rekaman untuk kategori ‘buka’ dan 100 rekaman untuk kategori ‘tutup’. Tujuan dari penelitian ini adalah untuk membangun model klasifikasi yang mampu mengenali perbedaan suara secara akurat menggunakan metode machine learning.

In [1085]:
import librosa
import soundfile as sf
import numpy as np
import os
import random
import string
import pandas as pd
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import joblib
from sklearn.svm import SVC


In [1086]:
buka_data_real = "./data_audio/buka_wav_asli/"

tutup_data_real = "./data_audio/tutup_wav_asli/"


In [1087]:
buka_file_real = os.listdir("./data_audio/buka_wav_asli")

tutup_file_real = os.listdir("./data_audio/tutup_wav_asli")

In [1088]:
def extract_mfcc(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc_mean

In [1089]:
all_data_buka = [buka_data_real  + real for real in buka_file_real]


all_data_tutup = [tutup_data_real  + real for real in tutup_file_real]

In [1090]:
dataset_buka = []

for buka in all_data_buka:
    a = extract_mfcc(buka)
    dataset_buka.append(list(a) + ["buka"])

dataset_buka = np.array(dataset_buka)
dataset_buka.shape

(100, 14)

In [1091]:
dataset_buka

array([['-658.1131', '75.941895', '2.840254', ..., '-4.299908',
        '-1.9714433', 'buka'],
       ['-616.4259', '82.20466', '-0.0047453945', ..., '-6.2080417',
        '-6.3981647', 'buka'],
       ['-685.40295', '59.30608', '0.6133957', ..., '-4.7687526',
        '-2.363253', 'buka'],
       ...,
       ['-656.3531', '60.979908', '1.346821', ..., '-5.529981',
        '-5.484186', 'buka'],
       ['-628.17676', '69.83497', '1.6306998', ..., '-6.217493',
        '-5.627337', 'buka'],
       ['-643.58984', '73.95612', '2.431623', ..., '-3.05748',
        '-4.7353325', 'buka']], shape=(100, 14), dtype='<U32')

In [1092]:
dataset_tutup = []

for tutup in all_data_tutup:
    a = extract_mfcc(tutup)
    dataset_tutup.append(list(a) + ["tutup"])

dataset_tutup = np.array(dataset_tutup)
dataset_tutup.shape

(100, 14)

In [1093]:
dataset_tutup

array([['-672.7006', '57.12414', '12.500899', ..., '-4.2370167',
        '-2.739796', 'tutup'],
       ['-675.3674', '64.59327', '16.245174', ..., '-6.4499946',
        '-5.179045', 'tutup'],
       ['-662.8768', '74.21501', '12.30772', ..., '-7.187595',
        '-4.8939285', 'tutup'],
       ...,
       ['-702.62054', '44.318558', '9.343956', ..., '-3.2685263',
        '-3.805813', 'tutup'],
       ['-615.64307', '72.36605', '11.764917', ..., '-7.270032',
        '-6.4610844', 'tutup'],
       ['-727.52496', '45.42344', '8.111155', ..., '-2.820191',
        '-2.6614997', 'tutup']], shape=(100, 14), dtype='<U32')

In [1094]:
columns = [f"mfcc_{i}" for i in range(len(dataset_buka[0]) - 1)] + ["label"]

dataset = pd.DataFrame(np.vstack([dataset_buka, dataset_tutup]), columns=columns)

dataset

,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,label
0,-658.1131,75.941895,2.840254,5.064627,3.7634497,9.87254,5.050689,5.5414486,-7.9472575,-1.5243582,-3.5832741,-4.299908,-1.9714433,buka
1,-616.4259,82.20466,-0.0047453945,1.814495,0.03942658,8.82973,6.0134845,3.662448,-5.199376,-3.7730248,-6.4048233,-6.2080417,-6.3981647,buka
2,-685.40295,59.30608,0.6133957,1.4330869,2.5678778,8.625678,3.003318,4.621652,-4.674938,-2.0727952,-3.3507373,-4.7687526,-2.363253,buka
3,-659.43036,61.924213,0.31243843,-0.23860456,2.5063918,9.683895,2.42775,5.6738477,-3.8325346,-1.3206859,-5.188299,-7.3316708,-3.2531664,buka
4,-664.09814,68.41928,4.7907863,5.5247965,2.6952608,8.024636,2.7801602,5.4909773,-4.6542587,-0.6678526,-2.0346162,-6.5158615,-3.8417077,buka
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-676.2904,38.25561,7.923041,4.0164785,-3.0596619,1.9310435,1.8121437,-1.6741729,-6.2673,-3.592635,-6.0317574,-2.259676,-2.666482,tutup
196,-672.73615,49.0395,11.746545,7.011384,-0.57286537,2.7780797,3.0674927,-1.9307669,-5.1297927,-4.3681145,-5.80346,-3.7468963,-3.442515,tutup
197,-702.62054,44.318558,9.343956,6.3822556,-1.4128219,2.3308258,1.531892,-1.2390318,-5.6697135,-3.9983752,-7.5267606,-3.2685263,-3.805813,tutup
198,-615.64307,72.36605,11.764917,9.645272,-3.224452,1.8182536,3.5944872,-1.5261397,-10.76896,-10.115143,-8.993052,-7.270032,-6.4610844,tutup


In [1095]:
dataset.to_csv("./dataset/dataset.csv")

In [1096]:
# Load dataset
dataset = pd.read_csv("./dataset/dataset.csv")

# Fitur dan label
X = dataset[[f"mfcc_{i}" for i in range(len(dataset_tutup[0]) - 1)]]
y = dataset["label"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.9, random_state=42, stratify=y
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

# Training
svm_model.fit(X_train_scaled, y_train)

# Prediksi
y_pred = svm_model.predict(X_test_scaled)


In [1097]:
accuracy_score(y_test, y_pred)

1.0

In [1098]:
joblib.dump(svm_model, "svm_model.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

[MODEL SVM](./svm_model.pkl)  
[SCALER](./scaler.pkl)  
[STREAMLIT](https://23-087-suara-buka-tutup.streamlit.app/)